
### 🚀 Project 4: Neural Style Transfer (NST) in 5 Points

* **1. The Objective:** The goal is to blend two distinct images—a **Content Image** (the structure) and a **Style Image** (the texture)—to create a new, AI-generated artwork that looks like a painting of the original photo.
* **2. How the AI "Sees" Art:** The project uses a **Convolutional Neural Network (CNN)**. It recognizes that deep layers of the brain capture **Shapes and Objects** (Content), while the shallow layers capture **Colors and Patterns** (Style).
* **3. The Secret Math (Gram Matrix):** To capture "Style," we don't just look at pixels; we use a **Gram Matrix**. This calculates how different features (like "blue swirls" and "thick brushstrokes") relate to each other across the image, defining the artist's unique "fingerprint."
* **4. Fast Style Transfer (Magenta):** Instead of the original 2015 method that took hours to "optimize" one image, this project uses a **Feed-forward Style Prediction Network**. This allows the AI to apply any artistic style to any photo in a single "glance" (inference pass), making it near-instant.
* **5. Strategic Application (Transfer Learning):** By using a pre-trained model from **TensorFlow Hub**, we demonstrate **Transfer Learning**. We leverage a model that has already seen thousands of paintings, allowing us to generate professional-grade art without needing a supercomputer or a massive dataset.



In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image

# 1. LOAD THE PRE-TRAINED "ARTISTIC BRAIN"
# We use the Magenta model from TensorFlow Hub.
# This model is a "Fast Style Transfer" model—it doesn't need to be trained;
# it already knows how to extract style and content.
print("Loading the Style Transfer Model...")
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

def load_img(path_to_img):
    """
    Helper Function: Loads an image and turns it into a 'Tensor' (a math grid).
    AI models can't see 'JPGs'; they only see numbers between 0 and 1.
    """
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3) # Convert to RGB
    img = tf.image.convert_image_dtype(img, tf.float32) # Scale pixels to [0, 1]

    # Resize so the image isn't too large for the memory
    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim
    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :] # Add a 'batch' dimension (1, height, width, 3)
    return img

def tensor_to_image(tensor):
    """
    Helper Function: Converts the AI's mathematical output back into a
    standard image that humans can look at.
    """
    tensor = tensor * 255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor) > 3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

# 2. SELECT YOUR CONTENT AND STYLE
# Content = The photo (structure) | Style = The painting (texture)
# You can replace these URLs with your own local file paths!
content_url = 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg'
style_url = 'https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg'

content_path = tf.keras.utils.get_file('content.jpg', content_url)
style_path = tf.keras.utils.get_file('style.jpg', style_url)

content_image = load_img(content_path)
style_image = load_img(style_path)

# 3. RUN THE STYLE TRANSFER
# The model takes the Content and Style as inputs and generates the artwork.
# It uses 'Forward Pass' inference to apply the style bottleneck.
print("Applying Artistic Style... please wait.")
stylized_image_tensor = hub_model(tf.constant(content_image), tf.constant(style_image))[0]
final_artwork = tensor_to_image(stylized_image_tensor)

# 4. VISUALIZE THE RESULTS
plt.figure(figsize=(16, 6))

plt.subplot(1, 3, 1)
plt.imshow(np.squeeze(content_image))
plt.title('Source Content (The "What")')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(np.squeeze(style_image))
plt.title('Artistic Style (The "How")')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(final_artwork)
plt.title('AI Generated Masterpiece')
plt.axis('off')

plt.tight_layout()
plt.show()

# Optional: Save your artwork
# final_artwork.save("my_ai_art.jpg")